In [2]:
import sys
import os
import pandas as pd
import geopandas as gpd
import numpy as np
from utils import *
from tqdm import tqdm
tqdm.pandas()

In [3]:
# Specify the data folder paths where all datasets as stored
config = load_config()
incoming_data_path = config['paths']['incoming_data']
processed_data_path = config['paths']['data']

nbs_input_data_path = os.path.join(incoming_data_path,"nbs")
# The projection system for Jamaica
epsg_jamaica = 3448

In [4]:
# Specify the input layers we want to create buffers around and intersect
natural_layers = ["nsmdb-mangroves.gpkg",
                 "nsmdb-seagrass.gpkg",
                 "nsmdb-reefs_jan06_NEPA.gpkg"]
output_layer_name = ["mangroves",
                     "seagrass",
                     "corals"]
# Buffer distances in meters
buffer_distances = [100,250,500,1000]

In [5]:
# Create a for loop over the buffer distances and the layer 
# Add the buffer to the geometry of the layer 
# Write the result to a new geopackage file
for buffer in buffer_distances:
    for i,(input_layer,output_layer) in enumerate(list(zip(natural_layers,output_layer_name))):
        # Read a Geopackage layer or shapefile
        get_layer = gpd.read_file(os.path.join(nbs_input_data_path,
                                                input_layer))
        # Reproject to Jamaica projection system
        get_layer = get_layer.to_crs(epsg=epsg_jamaica)
        # Create buffer around geometry 
        get_layer["geometry"] = get_layer.progress_apply(
                        lambda x: x.geometry.buffer(buffer),
                        axis=1)
        get_layer.to_file(os.path.join(processed_data_path,
                                       "nbs",
                                       f"{output_layer}_buffer.gpkg"),
                                          layer=f"{buffer}m",
                                         driver="GPKG")

 97%|███████████████████████████████████████████████████████████████████████████████▏  | 643/666 [00:02<00:00, 171.34it/s]/Users/raghavpant/miniconda3/envs/py39/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:1990: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  result[:] = values
 98%|█████████████████████████████████████████████████████████████████████████████████▊ | 656/666 [00:34<00:02,  3.75it/s]/Users/raghavpant/miniconda3/envs/py39/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:1990: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  result[:] = values
 98%|█████████████████████████████████████████████████████████████████████████████████

In [8]:
# Intersect the different buffer layers to find common areas
# Buffer distances in meters
buffer_distances = [250,500,1000]
intersection_layers = ["mangroves",
                     "seagrass",
                     "corals"]
for buffer in buffer_distances:
    for i in range(len(intersection_layers)-1):
        name_i = intersection_layers[i] 
        for j in intersection_layers[i+1:]:
            name_j = j
            layer_i = gpd.read_file(os.path.join(processed_data_path,
                                       "nbs",
                                       f"{name_i}_buffer.gpkg"),
                                          layer=f"{buffer}m")
            layer_i.rename(columns={"ID":f"{name_i}_id"},inplace=True)
            layer_j = gpd.read_file(os.path.join(processed_data_path,
                                       "nbs",
                                       f"{name_j}_buffer.gpkg"),
                                          layer=f"{buffer}m")
            layer_j.rename(columns={"ID":f"{name_j}_id"},inplace=True)
            common_ij = gpd.sjoin(layer_i,layer_j,
                                  how="inner",
                                  predicate="intersects").reset_index()
            common_ij.rename(columns={"geometry":"layer_i_geometry"},
                             inplace=True)
            common_ij = pd.merge(common_ij,
                                 layer_j[[f"{name_j}_id","geometry"]],
                                 how="left",on=[f"{name_j}_id"])
            common_ij.rename(columns={"geometry":"layer_j_geometry"},
                             inplace=True)
            common_ij["geometry"] = common_ij.progress_apply(
                lambda x:x.layer_i_geometry.intersection(x.layer_j_geometry)
                ,axis=1)
            common_ij.drop(["layer_i_geometry","layer_j_geometry"],
                           axis=1,
                           inplace=True)
            common_ij = gpd.GeoDataFrame(common_ij,
                             geometry="geometry",
                             crs=f"EPSG:{epsg_jamaica}")
            common_ij.to_file(os.path.join(processed_data_path,
                            "nbs",
                            f"{name_i}_{name_j}_intersections.gpkg"),
                            layer=f"{buffer}m",
                            driver="GPKG")

  0%|                                                                                             | 0/375 [00:00<?, ?it/s]/Users/raghavpant/miniconda3/envs/py39/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:1990: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  result[:] = values
100%|█████████████████████████████████████████████████████████████████████████████████| 375/375 [00:00<00:00, 6379.05it/s]


CPLE_AppDefinedError: b'failed to prepare SQL: SELECT "fid", ST_MinX("geom"), ST_MaxX("geom"), ST_MinY("geom"), ST_MaxY("geom") FROM "250m" WHERE "geom" NOT NULL AND NOT ST_IsEmpty("geom")'

Exception ignored in: 'fiona._shim.gdal_flush_cache'
Traceback (most recent call last):
  File "fiona/_err.pyx", line 201, in fiona._err.GDALErrCtxManager.__exit__
fiona._err.CPLE_AppDefinedError: b'failed to prepare SQL: SELECT "fid", ST_MinX("geom"), ST_MaxX("geom"), ST_MinY("geom"), ST_MaxY("geom") FROM "250m" WHERE "geom" NOT NULL AND NOT ST_IsEmpty("geom")'


RuntimeError: GDAL Error: sqlite3_exec(CREATE TABLE "250m" ( "fid" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, "geom" GEOMETRY, "index" INTEGER, "TARGET" TEXT, "CONDITION" REAL, "GEOCLIMATI" TEXT, "MARXAN_COD" REAL, "HECTARES_left" REAL, "mangroves_id" REAL, "TYPE" TEXT, "AREA" REAL, "PERIMETER" REAL, "Shape_Leng_left" REAL, "Shape_Area_left" REAL, "Parish" TEXT, "index_right" INTEGER, "OBJECTID" INTEGER, "SOURCE_TYP" TEXT, "MXAN_CODE" REAL, "AREA_METER" REAL, "HECTARES_right" REAL, "seagrass_id" REAL, "Shape_Leng_right" REAL, "PARISH" TEXT, "Shape_Length" REAL, "Shape_Area_right" REAL)) failed: duplicate column name: PARISH 
 
 Failed to write record: {'id': '0', 'type': 'Feature', 'properties': {'AREA': 23274.596, 'AREA_METER': 64787.477, 'CONDITION': 1.0, 'GEOCLIMATI': 'Moist_Limestone', 'HECTARES_left': 2.327, 'HECTARES_right': 6.479, 'MARXAN_COD': 403.0, 'MXAN_CODE': 405.0, 'OBJECTID': 150, 'PARISH': 'St. James', 'PERIMETER': 667.507, 'Parish': 'St. James', 'SOURCE_TYP': 'Coastal 